# Лабораторная работа XI.9.3б

## Задание

Построить алгоритм метода пристрелки для вычисления решения слудющей нелинейной задачи:

$$
\begin{cases}
  y'' - x \sqrt{y} = 0, \hspace{1cm} 0 \leqslant x \leqslant 1 \\
  y(0) = 0, \hspace{0.5cm} \int\limits_0^1 y(x) dx = 1.
\end{cases}
$$

In [42]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import odeint

Напишем простейшую функцию для численного интегрирования

In [43]:
def integrate(x, y):
  result = 0;
  for i in range(len(x) - 1):
    result += (y[i][0] + y[i+1][0]) / 2 * (x[i+1] - x[i])
  return result

Напишем функцию поиска наиболее точного $\alpha$. Для этого рассчитаем примерное значение, а затем будем постепенно приближать значение полученного интеграла к заданному в условии, циклически меняя $\alpha$ вверх и вниз и уменьшая шаг его варьирования.

In [44]:
def getApproximateAlpha(equation, x0, xk, y0, integral, epsilon = 0.0001, grid_resolution = 100000):
  approx_alpha = 2 * integral / (xk - x0)**2 - 2 * y0 / (xk - x0) # Getting approximate alpha
  alpha_step = approx_alpha / 10  # Using value of alpha decreased in 10 times to calculate real alpha
  stepping_direction = 0
  precision_reached = False

  x = np.linspace(x0, xk, grid_resolution)
  y_start = [y0, approx_alpha]
  y = odeint(equation, y_start, x)
  actual_integral = integrate(x, y)
  delta = integral - actual_integral
  redirections_count = 0
  
  if abs(delta) < epsilon:
    print(f'Founded alpha with setted accuracy = {epsilon} in {redirections_count} steps: \nalpha = {approx_alpha}, integral = {actual_integral} ({integral} requested)')
    precision_reached = True

  else:
    stepping_direction = delta / abs(delta)

  while not precision_reached and redirections_count < 100:
    approx_alpha += stepping_direction * alpha_step
    y_start = [y0, approx_alpha]
    y = odeint(equation, y_start, x)
    actual_integral = integrate(x, y)
    delta, previous_delta = integral - actual_integral, delta
    
    if abs(delta) < epsilon:
      print(f'Founded alpha with setted accuracy = {epsilon} in {redirections_count} steps: \nalpha = {approx_alpha}, integral = {actual_integral} ({integral} requested)')
      precision_reached = True

    elif delta / previous_delta < 0:  # if we jumped over target value, changing direction and decreasing step
      stepping_direction = -stepping_direction
      alpha_step /= 2
      redirections_count += 1

  return approx_alpha

Так же напишем функцию, которая будет строить график функции по полученному значению $\alpha$

In [45]:
def drawPlot(equation, x0, xk, y0, integral, alpha, grid_resolution = 100000):
  approx_alpha = 2 * integral / (xk - x0)**2 - 2 * y0 / (xk - x0)

  x = np.linspace(x0, xk, grid_resolution)
  y_start_approximate = [y0, approx_alpha]
  y_start_precise = [y0, alpha]

  y_approximate = odeint(equation, y_start_approximate, x)[:, 0]
  y_precise = odeint(equation, y_start_precise, x)[:, 0]

  plt.figure(figsize = [12, 8])
  plt.plot(x, y_approximate, color = 'blue', label = f'Approximate plot ($\\alpha_{{approx}} = {approx_alpha}$)')
  plt.plot(x, y_precise, color = 'red', label = f'Precise plot ($\\alpha_{{prec}} = {alpha}$)')
  plt.grid()
  plt.legend()
  plt.title(f'Plots for function "{equation.__name__}"')
  plt.xlabel('$x$')
  plt.ylabel('$y$')

Теперь внесем задание в код и решим задачу с помощью изготовленных функций.

In [46]:
def task(y, x):
  return np.array([y[1], -x * np.sqrt(y[0])])

x0 = 0
y0 = 0
xk = 1
integral = 1
epsilon = 0.0000001

alpha = getApproximateAlpha(task, x0, xk, y0, integral, epsilon = epsilon)
drawPlot(task, x0, xk, y0, integral, alpha)

1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in sqrt
  


KeyboardInterrupt: ignored